1. скачать данные

Metadata includes descriptions, price, sales-rank, brand info, and co-purchasing links

In [ ]:
! wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Pet_Supplies.json.gz

--2021-12-12 15:44:56--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Pet_Supplies.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284871978 (272M) [application/octet-stream]
Saving to: ‘meta_Pet_Supplies.json.gz’

meta_Pet_Supplies.j 100%[===================>] 271.67M  20.4MB/s    in 14s     

2021-12-12 15:45:10 (19.6 MB/s) - ‘meta_Pet_Supplies.json.gz’ saved [284871978/284871978]



In [ ]:
import pandas as pd
import gzip
from tqdm.notebook import tqdm
import json
from string import punctuation, digits
import numpy as np
import re

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in tqdm(parse(path)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

# df = getDF('Pet_Supplies.json.gz')# .iloc[:200000]
meta = getDF('meta_Pet_Supplies.json.gz')

0it [00:00, ?it/s]

теперь таблица с отзывами

In [ ]:
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz

--2021-12-06 14:33:37--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36119369 (34M) [application/x-gzip]
Saving to: ‘reviews_Pet_Supplies_5.json.gz’

reviews_Pet_Supplie 100%[===================>]  34.45M  32.5MB/s    in 1.1s    

2021-12-06 14:33:38 (32.5 MB/s) - ‘reviews_Pet_Supplies_5.json.gz’ saved [36119369/36119369]



In [1]:
import json

In [ ]:
# import gzip

In [2]:
import pandas as pd

In [ ]:
# dflist = []
# arch = gzip.open('reviews_Pet_Supplies_5.json.gz', 'rb')
# for jsn in arch:
#     dflist.append(json.loads(jsn))

# df = pd.DataFrame(dflist)

In [ ]:
# df.to_csv('/content/drive/My Drive/pet_reviews.csv')

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
df = pd.read_csv('/content/drive/My Drive/pet_reviews.csv')

сопоставление метаданных и таблицы с отзывами по колонке asin - ID of the product

In [ ]:
revs = df.merge(meta, on='asin')

In [ ]:
df = revs[['reviewText', 'title', 'brand', 'main_cat']]
df = df.drop_duplicates(subset=['reviewText'])
df.to_csv('/content/drive/My Drive/pet_revs_forhw.csv', index=False)

# загрузка с диска готовых данных

In [1]:
import json
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
df = pd.read_csv('/content/drive/My Drive/pet_revs_forhw.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94335 entries, 0 to 94334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  94334 non-null  object
 1   title       94335 non-null  object
 2   brand       94322 non-null  object
 3   main_cat    94335 non-null  object
dtypes: object(4)
memory usage: 2.9+ MB


In [5]:
df.head()

,reviewText,title,brand,main_cat
0,I purchased the Trilogy with hoping my two cat...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies
1,There are usually one or more of my cats watch...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies
2,I bought the triliogy and have tested out all ...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies
3,My female kitty could care less about these vi...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies
4,"If I had gotten just volume two, I would have ...","Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies


In [ ]:
for el in df['reviewText'].iloc[:50]:
  print(el)

I purchased the Trilogy with hoping my two cats, age 3 and 5 would be interested.  The 3 yr old cat was fascinated for about 15 minutes but when the same pictures came on, she got bored.  The 5 year old watched for about a few minutes but then walked away. It is possible that because we have a wonderful courtyard full of greenery and trees and one of my neighbors has a bird feeder, that there is enough going on outside that they prefer real life versus a taped version.  I will more than likely pass this on to a friend who has cats that don't have as much wildlife to watch as mine do.
There are usually one or more of my cats watching TV and staying out of trouble when this DVD is playing.  They seem to like the mice and birds the most and maybe go a little less stir crazy being inside all the time.
I bought the triliogy and have tested out all the DVDs.  It appears that volume 2 is the most well received of the three and the one I would recommend.  It's funny to watch my cat watch it bc

## (3 балла) Предложите 3 способа найти упоминания товаров в отзывах

1. rule-based подход: написать правила и шаблоны с помощью natasha/yargy. для этого понадобятся тексты отзывов и названия товаров из метадаты в качестве дополнительных ключевых слов, которые можно использовать при составлении правил (раз уж они доступны для скачивания). недостатки: подразумевает много ручной работы, поэтому не очень подходит для моих данных, которые довольно разнообразны. есть еще вариант выделить подкатегории среди pet supplies, например, еда, игрушки, dvds (не знала, что такие существуют, лол) и тогда возможно легче будет написать правила в зависимости от категорий. но все равно очень сложно перечислить все возможные варианты шаблонов. достоинства метода: при хорошо составленных правилах результат должен быть точнее, чем при совсем автоматическом выделении NE

2. Classification-based подход: разметить часть корпуса, обучить на ней нейронную сеть, которая определяет, является ли NE данная ей на вход последовательность символов. недостатки: нужно разметить корпус это ручная работа. достоинства метода: будучи обучена на отзывах конкретной категории, модель будет подстроена под нужные данные. модель должна работать лучше чем rule-based подход на новых данных. 
понадобятся тексты отзывов, названия товаров из метадаты в качестве помощи аннотаторам train данных



3. выделить ключевые слова и коллокации, отфильтровать. понадобится колонка title с названием товара, логично предположить, что там сконцентрированы нужные нам ключевые слова. достоинства: простой метод, без ручной работы. недостатки: не задействованы сами тексты отзывов, в которых можно найти синонимы названий товаров, которые не встречаются в title

## (2 балла) Реализуйте один из предложенных вами способов.

In [6]:
from string import punctuation
punctuation += "...-"

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


tfidf с учетом биграмм и триграмм

ngram_range=(1,3)

в названиях к ревью собственно уже собраны ключевые слова, выделим из них самые важные

In [8]:
import re

def delete_trash(text):
    text = str(text)
    for ch in punctuation:
        text = text.replace(ch, "").lower()
    for ch in '0123456789':
        text = text.replace(ch, "").lower()
    # delete non-english characters
    for ch in text:
        if not ch.isascii():
            text = text.replace(ch, "")
    return re.sub("\s+", " ", text.strip())

In [9]:
df['clean'] = df['title'].apply(delete_trash)

In [10]:
df.head()

,reviewText,title,brand,main_cat,clean
0,I purchased the Trilogy with hoping my two cat...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies,cat sitter dvd trilogy vol vol and vol
1,There are usually one or more of my cats watch...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies,cat sitter dvd trilogy vol vol and vol
2,I bought the triliogy and have tested out all ...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies,cat sitter dvd trilogy vol vol and vol
3,My female kitty could care less about these vi...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies,cat sitter dvd trilogy vol vol and vol
4,"If I had gotten just volume two, I would have ...","Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,Pet Supplies,cat sitter dvd trilogy vol vol and vol


In [105]:
# df['clean_reviews'] = df['reviewText'].apply(delete_trash)

In [11]:
titles = set(df['clean'].unique())

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
vectorizer=TfidfVectorizer(ngram_range=(1,3),max_features=1000,stop_words=stop)
X = vectorizer.fit_transform(titles)

In [14]:
X.shape

(4820, 1000)

In [15]:
X = X.toarray()

In [16]:
feats = vectorizer.get_feature_names_out()

In [18]:
import numpy as np

In [19]:
n = 1
kws_ind = np.unique((-X).argsort()[:,:n]) # top n key word for every text, then unique

In [20]:
len(kws_ind)

888

In [21]:
kws = feats[kws_ind]

In [22]:
kws[:100]

array(['accessory', 'action', 'adult dog', 'adult dry cat', 'adult wet',
       'advanced', 'aid', 'air', 'air pump', 'airline', 'algae',
       'allnatural', 'american', 'amp', 'amp cat', 'amp chewy',
       'amp chicken', 'amp odor remover', 'amp rice', 'animal', 'animals',
       'anti', 'api', 'aqua', 'aquarium', 'aquariums', 'arm',
       'arm amp hammer', 'assorted', 'assorted colors', 'attachment',
       'auto', 'automatic', 'available', 'bacon', 'bag', 'bags', 'ball',
       'ball dog', 'ball dog toy', 'balls', 'bark', 'bark collar',
       'bark control', 'bath', 'bear', 'bed', 'bedding', 'beef', 'beige',
       'belt', 'bergan', 'big', 'bio', 'bird', 'bird food', 'bird toy',
       'birds', 'biscuits', 'bites', 'bitesized', 'bitesized chews',
       'black', 'blade', 'blend', 'blue', 'blue buffalo', 'bone',
       'bone dog', 'bones', 'bonus', 'booda', 'booster', 'bottle', 'bowl',
       'box', 'braided', 'brand', 'breath', 'breed', 'brown',
       'brown rice', 'browns', 'b

фильтрация

In [23]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [24]:
final = set()

In [25]:
for el in kws:
    if ' ' not in el:
        doc = nlp(el)
        pos = [token.pos_ for token in doc]
        if pos[0] != 'NOUN': # если униграмма - должно быть существительное
            continue
        if el in ['cat', 'cats', 'dog', 'dogs']: # возможное улучшение - взять норм список животных
            continue
    
    final.add(el)

In [26]:
len(final)

468

In [ ]:
final

## (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

In [37]:
len(titles)

4820

In [ ]:
temp = [el.split() for el in titles]
temp[:10]

на биграммах лучшие результаты показывает t score

In [42]:
import nltk
from nltk.collocations import *

# тк до этого я собирала 1, 2 и 3 граммы теперь понадобятся 4 граммы
bigram_measures = nltk.collocations.BigramAssocMeasures()

finder2 = BigramCollocationFinder.from_documents([el.split() for el in titles])

# frequency filter
finder2.apply_freq_filter(2)

print('pmi')
print(finder2.nbest(bigram_measures.pmi, 10))
print('likelihood_ratio')
print(finder2.nbest(bigram_measures.likelihood_ratio, 10))
print('student_t')
print(finder2.nbest(bigram_measures.student_t, 10))


pmi
[('abe', 'chemipure'), ('au', 'jus'), ('belly', 'band'), ('clinically', 'proven'), ('dogwidgets', 'dw'), ('ecosphere', 'closed'), ('evriholder', 'furemover'), ('figure', 'eight'), ('hdc', 'yaky'), ('hearrrt', 'attack')]
likelihood_ratio
[('for', 'dogs'), ('pack', 'of'), ('outward', 'hound'), ('dog', 'toy'), ('for', 'cats'), ('zoo', 'med'), ('stainless', 'steel'), ('cat', 'food'), ('grain', 'free'), ('dog', 'treats')]
student_t
[('for', 'dogs'), ('dog', 'toy'), ('dog', 'food'), ('cat', 'food'), ('for', 'cats'), ('dog', 'treats'), ('pack', 'of'), ('dry', 'dog'), ('cat', 'toy'), ('cat', 'litter')]


на триграммах снова pmi показывает самый странный результат. лучше всех сработало likelihood_ratio

In [43]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder3 = TrigramCollocationFinder.from_documents([el.split() for el in titles])
finder3.apply_freq_filter(2)
print('pmi')
print(finder3.nbest(trigram_measures.pmi, 10))
print('likelihood_ratio')
print(finder3.nbest(trigram_measures.likelihood_ratio, 10))
print('student_t')
print(finder3.nbest(trigram_measures.student_t, 10))

pmi
[('enterprises', 'abe', 'chemipure'), ('notification', 'adhesive', 'backed'), ('pearilized', 'pebble', 'hungry'), ('ecotrition', 'snak', 'shak'), ('nongmo', 'project', 'verified'), ('nyantomo', 'purr', 'fect'), ('boyd', 'enterprises', 'abe'), ('beneful', 'prepared', 'meals'), ('pacific', 'pawdoodles', 'krinklers'), ('know', 'when', 'itrsquos')]
likelihood_ratio
[('dry', 'dog', 'food'), ('dry', 'cat', 'food'), ('for', 'dogs', 'and'), ('supplement', 'for', 'dogs'), ('for', 'dogs', 'ounce'), ('treats', 'for', 'dogs'), ('shampoo', 'for', 'dogs'), ('for', 'dogs', 'count'), ('sticks', 'for', 'dogs'), ('tablets', 'for', 'dogs')]
student_t
[('dry', 'dog', 'food'), ('dry', 'cat', 'food'), ('jw', 'pet', 'company'), ('for', 'dogs', 'and'), ('dogs', 'and', 'cats'), ('wet', 'cat', 'food'), ('dog', 'and', 'cat'), ('outward', 'hound', 'kyjen'), ('made', 'in', 'usa'), ('colors', 'may', 'vary')]
